# Загрузка и обучение модели для продуктовой болталки

## Загрузим необходимые библиотеки

In [1]:
import os
import numpy as np
import pandas as pd

import re
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

import annoy

from gensim.models import Word2Vec, FastText

from sklearn.feature_extraction.text import TfidfVectorizer

import joblib
import gzip
import pickle

from tqdm.notebook import tqdm
    
tqdm.pandas()

## Определим необходимые функции

In [2]:
# Функция предобработки текста
def preprocess_txt(line):
    # Удаляем HTML теги
    spls = re.sub('<[^<]+?>', ' ', line)
    # Удаляем URLs
    spls = re.sub(r'http\S+', ' ', spls)
    # Убираем специальные символы: избавляемся от всего, что не является "словами"
    #print(spls)
    spls = re.sub('[^a-zA-Zа-яА-ЯёЁ0-9\s!?-]', ' ', spls)
    # Удаляем лишние пробелы
    spls = re.sub('\s+', ' ', spls)
    # Удаляем знаки пунктуации
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    # Приводим слова к их нормальной форме
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    # Удаляем стоп слова
    #spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [3]:
def nan_useless_row(text):
    del_list = ['Экспресс-консультация детского нутрициолога по подбору добавок с iTAB', 'Полная консультация интегративного врача, Анна Горбачева',
                'Полная консультация детского нутрициолога', 'Полная консультация нутрициолога + 1 месяц сопровождения, Эльмира Измаилова',
                'Полная консультация нутрициолога, Светлана Корчажкина']
    for item in del_list:
        if item in text:
            return np.nan
    return text

In [4]:
# Основная функция преобразования текста в вектор х100
def get_embedded_text(txt, idfs, midf):
    text = preprocess_txt(txt)
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in text:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [5]:
def search_product(request, idfs, midf, ft_index_shop, num_row=5):
    vect_ft = get_embedded_text(request, idfs, midf)
    ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, num_row)
    for item in ft_index_shop_val:
        name, link = index_map_shop[item]
        print(name, link)

## Загрузим и предобработаем данные продуктов

In [6]:
# Создадим модель продуктовых данных
shop_data = pd.read_csv("itab_bad_products.csv", sep="|")

In [7]:
shop_data.head(3)

,Unnamed: 0,bad_name,link,Description,Regular_Price,Discounted_Price
0,0,"SuperMins, Витамин D3, жидкость, 10 мл",https://itab.pro/products/supermins-vitamin-d3...,Жидкий витамин D3 (2000 МЕ) от SuperMins - это...,670,463
1,1,"BioExpert, Биоактивный магниевый комплекс (5 ф...",https://itab.pro/products/bioexpert-bioaktivny...,Биоактивный магниевый комплекс (5 форм) от Bio...,2271,0
2,2,"Wolfsport+iTAB, Вкусный B-комплекс (вишня), шо...",https://itab.pro/products/wolfsportitab-vkusny...,Смотреть видео\nВ-комплекс от Wolfsport и iTAB...,3515,0


In [8]:
shop_data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [9]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [10]:
shop_data['text'] = shop_data['bad_name'] + " " + shop_data["Description"]
shop_data['text'] = shop_data['text'].progress_apply(preprocess_txt)

  0%|          | 0/1680 [00:00<?, ?it/s]

In [11]:
shop_data["class"] = 0

In [12]:
shop_data.head()

,bad_name,link,Description,Regular_Price,Discounted_Price,text,class
0,"SuperMins, Витамин D3, жидкость, 10 мл",https://itab.pro/products/supermins-vitamin-d3...,Жидкий витамин D3 (2000 МЕ) от SuperMins - это...,670,463,"[supermins, витамин, d3, жидкость, 10, мл, жид...",0
1,"BioExpert, Биоактивный магниевый комплекс (5 ф...",https://itab.pro/products/bioexpert-bioaktivny...,Биоактивный магниевый комплекс (5 форм) от Bio...,2271,0,"[bioexpert, биоактивный, магниевый, комплекс, ...",0
2,"Wolfsport+iTAB, Вкусный B-комплекс (вишня), шо...",https://itab.pro/products/wolfsportitab-vkusny...,Смотреть видео\nВ-комплекс от Wolfsport и iTAB...,3515,0,"[wolfsportitab, вкусный, bкомплекс, вишня, шот...",0
3,"BioExpert, Растительный мелатонин, жидкость, 3...",https://itab.pro/products/bioexpert-rastitelny...,Смотреть видеообзор.\nМелатонин от BioExpert -...,1500,1190,"[bioexpert, растительный, мелатонина, жидкость...",0
4,"Wolfsport+iTAB, Вкусный коллаген (фейхоа), шот...",https://itab.pro/products/wolfsportitab-vkusny...,Смотреть видео\nПолюбившийся всем вкусный колл...,3610,0,"[wolfsportitab, вкусный, коллаген, фейхоа, шот...",0


In [13]:
shop_data['bad_name'] = shop_data['bad_name'].progress_apply(nan_useless_row)

  0%|          | 0/1680 [00:00<?, ?it/s]

In [14]:
print(shop_data.shape)
shop_data.dropna(axis=0, inplace=True)
print(shop_data.shape)

(1680, 7)
(1675, 7)


In [15]:
shop_data.reset_index(inplace=True, drop=True)

In [16]:
shop_data.to_csv('bads_products_data_clean.csv')

In [17]:
print("Максимальное количество токенов в описании продукта: {}".format(np.max([len(text) for text in shop_data['text']])))
print("Среднее количество токенов в описании продукта: {}".format(np.mean([len(text) for text in shop_data['text']])))
print("Минимальное количество токенов в описании продукта: {}".format(np.min([len(text) for text in shop_data['text']])))

Максимальное количество токенов в описании продукта: 1384
Среднее количество токенов в описании продукта: 235.95164179104478
Минимальное количество токенов в описании продукта: 23


## Обучение модели FastText

#### Обучим модель FastText для быстрого поиска по продуктам

In [21]:
%%time
file_path_from = 'ft_model'
if not os.path.isfile(file_path_from):  
    sentences = [i for i in tqdm(shop_data['text']) if len(i) > 2]
    modelFT = FastText(sentences=sentences, vector_size=100, min_count=2, window=4)
    modelFT.train(corpus_iterable=sentences, epochs=30,
                  total_examples=modelFT.corpus_count,
                  total_words=modelFT.corpus_total_words)

CPU times: total: 0 ns
Wall time: 0 ns


#### Сохраним обученную модель

In [20]:
modelFT.save("ft_model")

NameError: name 'modelFT' is not defined

#### Загрузим обученную модель

In [22]:
modelFT = FastText.load("ft_model")

#### Получим индексы annoy

In [23]:
ft_index = annoy.AnnoyIndex(100 ,'angular')

#### Получим tfidf векторы продуктовых данных

In [24]:
products_text = [" ".join(val) for val in tqdm(shop_data['text'].values)]

  0%|          | 0/1675 [00:00<?, ?it/s]

In [25]:
tfidf_vect = TfidfVectorizer().fit(products_text)

In [26]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}

In [27]:
list(idfs.keys())[:5]

['supermins', 'витамин', 'd3', 'жидкость', '10']

#### Сохраним tfidf векторайзер

In [28]:
with open('tfidf.pkl', 'wb') as f:
    pickle.dump(tfidf_vect, f)

#### Создаем Индексы для продуктовых данных

In [25]:
file_path_from = 'bads_shop.ann'
if not os.path.isfile(file_path_from):  
    
    
    ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

    midf = np.mean(tfidf_vect.idf_)

    index_map_shop = {}
    counter = 0

    for i in tqdm(range(len(shop_data))):
        n_ft = 0
        index_map_shop[i] = (shop_data.loc[i, "bad_name"], shop_data.loc[i, "link"])
        vector_ft = np.zeros(100)
        for word in shop_data.loc[i, "text"]:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word] * idfs.get(word, midf)
                n_ft += idfs.get(word, midf)
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index_shop.add_item(counter, vector_ft)
        counter += 1

    ft_index_shop.build(10)
    ft_index_shop.save(file_path_from)

    file_path_from = 'index_bads_shop.pkl'
    if not os.path.isfile(file_path_from):  
    
        with open("index_bads_shop.pkl", "wb") as f:
            pickle.dump(index_map_shop, f)

  0%|          | 0/1675 [00:00<?, ?it/s]

#### Загрузим индексы 

In [26]:
midf = np.mean(tfidf_vect.idf_)

ft_index_shop = annoy.AnnoyIndex(100, 'angular')
ft_index_shop.load('bads_shop.ann') 

index_map_shop = pd.read_pickle("index_bads_shop.pkl")

#### Проверим поиск по продуктовому датасету

In [31]:
input_txt = "Витамин D"
search_product(input_txt, idfs, midf, ft_index_shop)

Restartbio, Витамин Д3 (5000 МЕ) + К2 (МК-7), капсулы, 90 шт. https://itab.pro/products/restartbio-vitamin-d3-5000-me-k2-mk-7-kapsuly-90-sht-1832/
Avicenna, D Max 5, Витамин Д3 (5000 МЕ), капсулы, 60 шт. https://itab.pro/products/avicenna-d-max-5-vitamin-d3-5000-me-kapsuly-60-sht-1977/
IN.OUT, Комплекс для волос и ногтей, таблетки, 90 шт https://itab.pro/products/inout-kompleks-dlya-volos-i-nogtey-tabletki-90-sht-784/
ORZAX, Океан «Daily One Energy» (витаминный комплекс), таблетки, 30 шт https://itab.pro/products/orzax-okean-daily-one-energy-vitaminnyy-kompleks-tabletki-30-sht-1378/
Алтайские традиции, Концентрат «Щитовидная железа» с экстрактом лапчатки и ламинарии +11 витаминов + 6 минералов, капсулы, 60 шт. https://itab.pro/products/altayskie-traditsii-kontsentrat-schitovidnaya-zheleza-s-ekstraktom-lapchatki-i-laminarii-11-vitaminov-6-mineralov-kapsuly-60-sht-1200/


In [38]:
input_txt = "Биоактивный магниевый комплекс"
search_product(input_txt, idfs, midf, ft_index_shop, num_row=15)

IPH Peptides FAST, Fast Telomers Care Экстрактно-пептидный комплекс, жевательные таблетки, 45 шт. https://itab.pro/products/iph-peptides-fast-fast-telomers-care-ekstraktno-peptidnyy-kompleks-zhevatelnye-tabletki-45-sht-2009/
Trace Minerals, Комплекс «Trace Mineral Drops» (для поддержания здоровья организма), жидкость, 15 мл https://itab.pro/products/trace-minerals-kompleks-trace-mineral-drops-dlya-podderzhaniya-zdorovya-organizma-zhidkost-15-ml-1812/
Trace Minerals, Комплекс «Trace Mineral Drops» (для поддержания здоровья организма), жидкость, 237 мл https://itab.pro/products/trace-minerals-kompleks-trace-mineral-drops-dlya-podderzhaniya-zdorovya-organizma-zhidkost-237-ml-1813/
IPH Peptides FAST, Fast Thyroids Support Экстрактно-пептидный комплекс, жевательные таблетки, 45 шт. https://itab.pro/products/iph-peptides-fast-fast-thyroids-support-ekstraktno-peptidnyy-kompleks-zhevatelnye-tabletki-45-sht-2010/
Эльзам, Комплекс «Vita Kids» Immuno (для повышения иммунитета детей), 10х10 мл htt

In [ ]:
input_txt = "Биоактивный магниевый комплекс"
search_product(input_txt, idfs, midf, ft_index_shop, num_row=15)